In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
text = ""
with open('sherlock.txt','r') as file:
    text = file.read()

In [4]:
text = text.split('\n')

In [5]:
x = lambda a : a.strip() != ''

In [6]:
list(filter(x,text))

['The Project Gutenberg EBook of The Adventures of Sherlock Holmes',
 'by Sir Arthur Conan Doyle',
 '(#15 in our series by Sir Arthur Conan Doyle)',
 'Copyright laws are changing all over the world. Be sure to check the',
 'copyright laws for your country before downloading or redistributing',
 'this or any other Project Gutenberg eBook.',
 'This header should be the first thing seen when viewing this Project',
 'Gutenberg file.  Please do not remove it.  Do not change or edit the',
 'header without written permission.',
 'Please read the "legal small print," and other information about the',
 'eBook and Project Gutenberg at the bottom of this file.  Included is',
 'important information about your specific rights and restrictions in',
 'how the file may be used.  You can also find out about how to make a',
 'donation to Project Gutenberg, and how to get involved.',
 '**Welcome To The World of Free Plain Vanilla Electronic Texts**',
 '**eBooks Readable By Both Humans and By Computers, 

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
tokenizer = Tokenizer()

In [9]:
tokenizer.fit_on_texts(text)

In [10]:
len(tokenizer.word_index)


31619

In [11]:
input_sequences = []
for sentence in text:
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [12]:
input_sequences

[[1, 400],
 [1, 400, 440],
 [1, 400, 440, 1404],
 [1, 400, 440, 1404, 2],
 [1, 400, 440, 1404, 2, 1],
 [1, 400, 440, 1404, 2, 1, 5556],
 [1, 400, 440, 1404, 2, 1, 5556, 2],
 [1, 400, 440, 1404, 2, 1, 5556, 2, 1171],
 [1, 400, 440, 1404, 2, 1, 5556, 2, 1171, 247],
 [19, 668],
 [19, 668, 3605],
 [19, 668, 3605, 13246],
 [19, 668, 3605, 13246, 11856],
 [2479, 5],
 [2479, 5, 102],
 [2479, 5, 102, 931],
 [2479, 5, 102, 931, 19],
 [2479, 5, 102, 931, 19, 668],
 [2479, 5, 102, 931, 19, 668, 3605],
 [2479, 5, 102, 931, 19, 668, 3605, 13246],
 [2479, 5, 102, 931, 19, 668, 3605, 13246, 11856],
 [1735, 485],
 [1735, 485, 35],
 [1735, 485, 35, 2617],
 [1735, 485, 35, 2617, 31],
 [1735, 485, 35, 2617, 31, 81],
 [1735, 485, 35, 2617, 31, 81, 1],
 [1735, 485, 35, 2617, 31, 81, 1, 317],
 [1735, 485, 35, 2617, 31, 81, 1, 317, 22],
 [1735, 485, 35, 2617, 31, 81, 1, 317, 22, 963],
 [1735, 485, 35, 2617, 31, 81, 1, 317, 22, 963, 4],
 [1735, 485, 35, 2617, 31, 81, 1, 317, 22, 963, 4, 2982],
 [1735, 485, 35

In [13]:
maxlen = max([len(sent) for sent in text])

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences[:5000], maxlen = maxlen, padding='pre')

In [15]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,  400],
       [   0,    0,    0, ...,    1,  400,  440],
       [   0,    0,    0, ...,  400,  440, 1404],
       ...,
       [   0,    0,    0, ..., 9207,  110,  852],
       [   0,    0,    0, ...,  110,  852,   14],
       [   0,    0,    0, ...,  852,   14,  653]], dtype=int32)

In [16]:
X = padded_input_sequences[:,:-1]

In [17]:
y = padded_input_sequences[:,-1]

In [18]:
X.shape,y.shape

((5000, 2540), (5000,))

In [19]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=len(tokenizer.word_index)+1)

In [20]:
y.shape

(5000, 31620)

In [21]:
from tensorflow.keras.models import Sequential

In [22]:
from tensorflow.keras.layers import Dense,LSTM,Embedding

In [23]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index),100,input_length=maxlen - 1))
model.add(LSTM(400))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(len(tokenizer.word_index)+1,activation='softmax'))

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2540, 100)         3161900   
                                                                 
 lstm (LSTM)                 (None, 400)               801600    
                                                                 
 dense (Dense)               (None, 128)               51328     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 31620)             4078980   
                                                                 
Total params: 8110320 (30.94 MB)
Trainable params: 8110320 (30.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.fit(X,y,epochs=100)

Epoch 1/100
157/157 [==============================] - 61s 336ms/step - loss: 7.5931 - accuracy: 0.0424
Epoch 2/100
157/157 [==============================] - 44s 284ms/step - loss: 6.2466 - accuracy: 0.0508
Epoch 3/100
157/157 [==============================] - 40s 255ms/step - loss: 6.0306 - accuracy: 0.0538
Epoch 4/100
157/157 [==============================] - 39s 251ms/step - loss: 5.7846 - accuracy: 0.0710
Epoch 5/100
157/157 [==============================] - 37s 238ms/step - loss: 5.5813 - accuracy: 0.0756
Epoch 6/100
157/157 [==============================] - 37s 235ms/step - loss: 5.4348 - accuracy: 0.0796
Epoch 7/100
157/157 [==============================] - 37s 238ms/step - loss: 5.3119 - accuracy: 0.0822
Epoch 8/100
157/157 [==============================] - 37s 235ms/step - loss: 5.2046 - accuracy: 0.0840
Epoch 9/100
157/157 [==============================] - 37s 239ms/step - loss: 5.0908 - accuracy: 0.0924
Epoch 10/100
157/157 [==============================] - 36s 232m

In [28]:
import numpy as np
new_text = "Sherlock"
for i in range(30):
    token_text = tokenizer.texts_to_sequences([new_text])[0]
      # padding
    padded_token_text = pad_sequences([token_text], maxlen=maxlen-1, padding='pre')
      # predict
    pos = np.argmax(model.predict(padded_token_text))
    word = list(tokenizer.word_index)[pos]
    new_text+=' ' + word
    print(new_text)

1/1 [==============================] - 0s 59ms/step
Sherlock called
1/1 [==============================] - 0s 59ms/step
Sherlock called they
1/1 [==============================] - 0s 160ms/step
Sherlock called they all
1/1 [==============================] - 0s 129ms/step
Sherlock called they all possibility
1/1 [==============================] - 0s 57ms/step
Sherlock called they all possibility and
1/1 [==============================] - 0s 57ms/step
Sherlock called they all possibility and was
1/1 [==============================] - 0s 51ms/step
Sherlock called they all possibility and was of
1/1 [==============================] - 0s 55ms/step
Sherlock called they all possibility and was of hopeless
1/1 [==============================] - 0s 55ms/step
Sherlock called they all possibility and was of hopeless on
1/1 [==============================] - 0s 60ms/step
Sherlock called they all possibility and was of hopeless on of
1/1 [==============================] - 0s 56ms/step
Sherlock call

'and'